Defining the Quic client and its functions.

In [1]:
import os
import socket
import struct
import math
import random
import sys

LOCAL_IP = '127.0.0.1'
LOCAL_PORT = 12345
PACKET_SIZE = 1024
READY_MESSAGE = b"READY" # Message sent by the server to indicate that it is ready for the next run (Identify new iteration)
FINISHED_MESSAGE = b"FINISHED" # Message sent by the server to indicate that it has finished all iterations

class QuicClient:
    
    # Initialize the client 
    def __init__(self):
        self.streams = []
        self.iteration = 0

    
    # Generate random data of a given size
    def generate_random_data(self, size):
        return os.urandom(size)

    
    # Generate a random file of a given size
    def generate_random_file(self, file_size, filename="random_file.txt"):
        with open(filename, "wb") as file:
            file.write(self.generate_random_data(file_size))


    # Create a ID for streams
    def create_streams(self, num_streams):
        self.streams = [i + 1 for i in range(num_streams)]

    
    # Send a file over multiple streams
    def send_file_over_streams(self, file_size, filename="random_file.txt"):
        with open(filename, "rb") as file:
            file_data = file.read()
            num_packets = math.ceil(file_size / PACKET_SIZE)
            for stream_id in self.streams:
                for i in range(num_packets):
                    start = i * PACKET_SIZE
                    end = min(start + PACKET_SIZE, file_size)
                    chunk = file_data[start:end]
                    self.send_data(stream_id, chunk)
                self.send_end_of_stream(stream_id)

    
    # Send data over a stream
    def send_data(self, stream_id, data):
        server_address = (LOCAL_IP, LOCAL_PORT)
        message = struct.pack("!I", stream_id) + data
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            sock.sendto(message, server_address)

    
    # Send end of stream message
    def send_end_of_stream(self, stream_id):
        server_address = (LOCAL_IP, LOCAL_PORT)
        message = struct.pack("!I", stream_id) + b"END"
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            sock.sendto(message, server_address)


    # Send initial message to the server with the number of streams and file size
    def send_initial_message(self, num_streams, file_size):
        initial_message = struct.pack("!II", num_streams, file_size)
        server_address = (LOCAL_IP, LOCAL_PORT)
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            sock.sendto(initial_message, server_address)


    # Wait for the server to send a 'Ready' message (identifying the next iteration)
    def wait_for_ready_message(self):
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            sock.bind((LOCAL_IP, LOCAL_PORT + 1))
            sock.settimeout(10)
            try:
                message, _ = sock.recvfrom(4096)
                if message == READY_MESSAGE:
                    print("Received 'Ready for next run' message from server.")
                    self.iteration += 1
            except socket.timeout:
                print("Timeout waiting for 'Ready for next run' message.")
                sys.exit(1)
                
    
    # Wait for the server to send a 'Finished' message (indicating that all iterations are done)
    def wait_for_finished_message(self):
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            sock.bind((LOCAL_IP, LOCAL_PORT + 1))
            sock.settimeout(10)
            try:
                message, _ = sock.recvfrom(4096)
                if message == FINISHED_MESSAGE:
                    print("Received 'Finished' message from server.")
                    self.iteration += 1
            except socket.timeout:
                print("Timeout waiting for 'Finished' message.")
                sys.exit(1)


    # Run the client
    def run(self):
        two_mb = 2 * 1024 * 1024
        one_mb = 1 * 1024 * 1024
        file_size = random.randint(one_mb, two_mb)
        self.generate_random_file(file_size)
        print(f"Sending a file of size {file_size} bytes.\n")
        while self.iteration < 5:
            num_streams = random.randint(1, 10)
            self.create_streams(num_streams)
            print(f"Iteration {self.iteration + 1}:\n")
            print(f"Sending file over {num_streams} streams.\n")
            self.send_initial_message(num_streams, file_size)
            self.send_file_over_streams(file_size)
            if self.iteration < 4:
                self.wait_for_ready_message()
            elif self.iteration == 4:
                self.wait_for_finished_message()
            elif self.iteration > 4:
                print(f"Invalid iteration {self.iteration}")
                os.remove("random_file.txt")
                break
        
        # Clean up
        os.remove("random_file.txt")

if __name__ == "__main__":
    client = QuicClient()
    client.run()


Sending a file of size 1183039 bytes.

Iteration 1:

Sending file over 5 streams.

Received 'Ready for next run' message from server.
Iteration 2:

Sending file over 9 streams.

Received 'Ready for next run' message from server.
Iteration 3:

Sending file over 8 streams.

Received 'Ready for next run' message from server.
Iteration 4:

Sending file over 1 streams.

Received 'Ready for next run' message from server.
Iteration 5:

Sending file over 3 streams.

Received 'Finished' message from server.


Creating a Quic Client and sending the file.

A random file is being generated of size 1 - 2 mega bytes.
Amount of streams are being generated (1-10)
The client then sends the file over to the receiver. Such that in each stream the same file is being sent once.

Clean up